In [1]:
from infopath.config import load_training_opt
from infopath.model_loader import load_model_and_optimizer
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [11]:
def generate_data(
    path="log_dir/c5b25604ab71d99a4c87243dc7fb79ce4dceb357/2023_10_19_15_2_54_par_rec/", 
    save=False,
    batch_size=200,
):
    opt = load_training_opt(path)
    print(opt.log_path, opt.lr)
    # opt.device = "cpu"
    model = load_model_and_optimizer(opt, reload=True, last_best="best")[0]
    if save:
        with torch.no_grad():
            spikes = model(torch.ones(batch_size), seed=0)[0]
            filt_model = model.filter_fun1(spikes)
        max_A2 = (
            filt_model[:, :, (model.rsnn.area_index == 1)].mean((2)).max(0)[0].cpu().numpy()
        )
        trial_type = ((max_A2 / 0.002) > 30) * 1
        print((trial_type * 1.0).mean())
        t = {0: "Miss", 1: "Hit"}
        trial_info = pd.read_csv(opt.datapath + "/PS000_20220404/trial_info")
        for i in range(batch_size):
            trial_info.loc[i, "trial_number"] = int(i)
            trial_info.loc[i, "reaction_time_jaw"] = 0.15
            trial_info.loc[i, "reaction_time_piezo"] = 0.15
            trial_info.loc[i, "stim"] = 4
            trial_info.loc[i, "trial_active"] = 1
            trial_info.loc[i, "trial_type"] = t[trial_type[i]]
            trial_info.loc[i, "trial_type"] = t[trial_type[i]]
            trial_info.loc[i, "trial_onset"] = 0.1 + i * 0.4
        trial_info["trial_number"] = trial_info["trial_number"].astype(int)
        trial_info.to_csv("datasets/ModelData/PS000_20220404/trial_info")
        for i, n in enumerate(model.neuron_index):
            spikes_neuron = np.where(spikes[:, :, i].T.flatten().cpu().numpy())[0] * 0.002
            np.save(
                f"datasets/ModelData/PS000_20220404/neuron_index_{int(n)}", spikes_neuron
            )
    return model, trial_info

model, trial_info = generate_data(save=True, batch_size=400)

log_dir/c5b25604ab71d99a4c87243dc7fb79ce4dceb357/2023_10_19_15_2_54_par_rec 0.001
0.85


In [10]:
trial_info

,Unnamed: 0,trial_number,reaction_time_jaw,reaction_time_piezo,stim,trial_type,trial_active,trial_onset,jaw_trace,tongue_trace,whisker_angle,completed_trials,video_onset,video_offset
0,0.0,0.0,0.15,0.15,4.0,Hit,1.0,0.1,NaN,NaN,NaN,NaN,2.0,0.0
1,1.0,1.0,0.15,0.15,4.0,Hit,1.0,0.5,NaN,NaN,NaN,NaN,2.0,0.0
2,2.0,2.0,0.15,0.15,4.0,Hit,1.0,0.9,NaN,NaN,NaN,NaN,2.0,0.0
3,3.0,3.0,0.15,0.15,4.0,Hit,1.0,1.3,NaN,NaN,NaN,NaN,2.0,0.0
4,4.0,4.0,0.15,0.15,4.0,Hit,1.0,1.7,NaN,NaN,NaN,NaN,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,NaN,395.0,0.15,0.15,4.0,Hit,1.0,158.1,NaN,NaN,NaN,NaN,NaN,NaN
396,NaN,396.0,0.15,0.15,4.0,Hit,1.0,158.5,NaN,NaN,NaN,NaN,NaN,NaN
397,NaN,397.0,0.15,0.15,4.0,Hit,1.0,158.9,NaN,NaN,NaN,NaN,NaN,NaN
398,NaN,398.0,0.15,0.15,4.0,Hit,1.0,159.3,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
opt = load_training_opt("log_dir/c5b25604ab71d99a4c87243dc7fb79ce4dceb357/2023_10_20_9_50_0_par_rec/")
opt.device = "cpu"
model_clone = load_model_and_optimizer(opt,reload=True,last_best="last")[0]
paths = os.listdir(opt.log_path)
paths = [int(p.split("_")[-1].split(".")[0]) for p in paths if "activity" in p]
step = max(paths)
# with torch.no_grad():
#     spikes_clone = model_clone(torch.ones(200))[0]
#     filt_model_clone = model_clone.filter_fun1(spikes_clone)
    
plt.plot(model_clone.rsnn._w_in.detach().cpu().numpy().mean(1), alpha=0.5)
plt.plot(model.rsnn._w_in.detach().cpu().numpy().mean(1), alpha=0.5)
plt.savefig(f"par_rec_comp/compare_w_in_step{step}.png")
plt.figure()
plt.plot(model_clone.rsnn._w_rec.detach().cpu().numpy().mean((0,1)), alpha=0.5)
plt.plot(model.rsnn._w_rec.detach().cpu().numpy().mean((0,1)), alpha=0.5)
plt.savefig(f"par_rec_comp/compare_w_rec_step{step}.png")